In [2]:
# Don't close visdom or Jupyter

from subprocess import Popen
import subprocess, shlex

def run_command(command, log=True):
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE, shell=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            if log:
                print(output.strip())
            else:
                yield(output.strip())
    rc = process.poll()
    return rc

def kill_gpu_hogs():
        pid = os.getpid()

        # All python commands - this works a little better, but will kill visdom
        find_processes_command = f"pgrep -fl python"
        find_processes_command = f"ps all" #3 | grep python"
        command = find_processes_command # + f" | awk '!/{pid}/{{print $1}}' " #| xargs kill"

        processes = run_command(command, log=False)
        for p in processes:
            print(p)
    
    
#kill_gpu_hogs()

In [34]:
# subprocess.check_output([SCRIPT, "-d", date], shell=True).
import os, signal
exclusion_words = "visdom", "jupyter"
find_processes_command = f"ps all | grep python" #3 | grep python"
x = subprocess.check_output([find_processes_command], shell=True)
all_python_processes = x.decode().split("\n")[:-1]
for process in all_python_processes:
    print(process)
    if not any([ew in process for ew in exclusion_words]):
        print(f"killing {process}")
        os.kill(int(process.split()[2]), signal.SIGTERM)


0  1000  42420  33382  20   0 350936 89844 ep_pol Sl+  pts/12     0:05 /home/taylor/anaconda3/bin/python /home/taylor/anaconda3/bin/jupyter-lab
0  1000  47361  47181  20   0 2193500 179712 ep_pol S+ pts/15     0:15 python -m visdom.server -p 8080


In [46]:
from subprocess import check_output, Popen
def kill_gpu_hogs():
    ## Try to kill just nvidia ones first; ask before killing everything; try to restart Visdom
    if True:
        print("Killing GPU hogs")
        ## KILL ALL OTHER PYTHON SCRIPTS
        pid = os.getpid()
        if False:
            # All GPU processes
            # find_processes_command = "nvidia-smi | sed -n 's/|\s*[0-9]*\s*\([0-9]*\)\s*.*/\1/p' | sort | uniq | sed '/^\$/d'"
            # All python commands - this works a little better, but will kill visdom
            find_processes_command = f"pgrep -fl python"
            command = find_processes_command + f" | awk '!/{pid}/{{print $1}}' | xargs kill"
            result = Popen(command, shell=True)
        else:
            exclusion_words = "visdom", "jupyter"
            find_processes_command = f"ps all | grep python"  + f" | awk '!/{pid}/'"
            x = check_output([find_processes_command], shell=True)
            all_python_processes = x.decode().split("\n")[:-1]
            for process in all_python_processes:
                if not any([ew in process for ew in exclusion_words]):
                    print(f"killing {process}")
                    pid = int(process.split()[2])
                    os.kill(pid, signal.SIGTERM)
        return
kill_gpu_hogs()

Killing GPU hogs
killing 0  1000  51445  51427  20   0  39408  7356 poll_s S+   pts/19     0:00 python
